In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!pip install keras-tuner

     |████████████████████████████████| 61kB 2.2MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.1-cp36-none-any.whl size=73200 sha256=94be1df7a964a051d7a20c513770a31387ee7114babd573c2ab63903dba04d2d
  Stored in directory: /root/.cache/pip/wheels/b9/cc/62/52716b70dd90f3db12519233c3a93a5360bc672da1a10ded43
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15356 sha256=506c45a8ad8c727415569aa33b158e7672ef99d8ff25c7a3c9b126d044f965a1
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [0]:
!pip install black==19.10b0
!pip install loguru==0.4.0

     |████████████████████████████████| 102kB 2.7MB/s 
     |████████████████████████████████| 747kB 10.7MB/s 
     |████████████████████████████████| 61kB 2.2MB/s 
     |████████████████████████████████| 102kB 5.2MB/s 
  Created wheel for contextvars: filename=contextvars-2.4-cp36-none-any.whl size=7666 sha256=6cd8c31ada293992a5f7d01762ff3a35472b4b8eebf1d5fad159dcd924f7db71
  Stored in directory: /root/.cache/pip/wheels/a5/7d/68/1ebae2668bda2228686e3c1cf16f2c2384cea6e9334ad5f6de
Successfully built contextvars


In [0]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate, Dropout
from tensorflow.keras.models import Model
from kerastuner import HyperModel
from keras.layers.normalization import BatchNormalization
from keras.layers.pooling import MaxPooling2D
from keras.layers.core import Lambda, Flatten, Dense

from keras.engine.topology import Layer
from keras.regularizers import l2
from keras import backend as K

K.set_image_data_format('channels_last')

In [0]:
#importing required library
import keras
import os
import glob
import pandas as pd
import numpy as np
import cv2

In [0]:
#preprocessing the image data
file_dir1 = '/content/drive/My Drive/siamese_net_internship/input1/*'
file_dir2 = '/content/drive/My Drive/siamese_net_internship/input2/*'
files = glob.glob(file_dir1)
print(len(files))
x1 = []
for f1 in files:
      img_dir = os.path.join(f1)
  #print('img_dir',img_dir)
      images = glob.glob(img_dir)
      for img in images:
        img_temp = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
        img_temp = cv2.resize(img_temp,(500,200))
        img_temo = img_temp/255
        x1.append(img_temp)
import numpy as np
x1 = np.array(x1)
print(x1.shape)

250
(250, 200, 500)


In [0]:
#%%
files = glob.glob(file_dir2)
x2 = []
for f1 in files:
  img_dir = os.path.join(f1)
  images = glob.glob(img_dir)
  
  for img in images:
    img_temp = cv2.imread(img,cv2.IMREAD_GRAYSCALE)
    img_temp = cv2.resize(img_temp,(500,200))
    img_temo = img_temp/255
    x2.append(img_temp)
import numpy as np
x2 = np.array(x2)
print(x2.shape)

(250, 200, 500)


In [0]:
ylabel = '/content/drive/My Drive/siamese_net_internship/ylabel.csv'
y = pd.read_csv(ylabel)



In [0]:
from sklearn.utils import shuffle
x1,x2,y = shuffle(x1,x2,y,random_state=2)

# splitting dataset into train test
x1_train = x1[0:180]
x1_train = x1_train.reshape(180,200,500,1)
x1_val = x1[180:210]
x1_val = x1_val.reshape(30,200,500,1)

x2_train = x2[0:180]
x2_train = x2_train.reshape(180,200,500,1)
x2_val = x2[180:210]
x2_val = x2_val.reshape(30,200,500,1)

x1_test = x1[210 :]
x1_test = x1_test.reshape(40,200,500,1)
x2_test = x2[210 :]
x2_test = x2_test.reshape(40,200,500,1)

y_train = y[0:180]
y_val = y[180:210]
y_test = y[210 :]

In [0]:
  
from kerastuner import HyperModel
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Flatten, MaxPooling2D


class CNNHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        input_shape = (200,500,1)
        left_input=Input(input_shape)
        right_input=Input(input_shape)
        model = keras.Sequential()
        model.add(
            Conv2D(
                filters=16,
                kernel_size=3,
                activation="relu",
                input_shape=self.input_shape,
            )
        )
        model.add(Conv2D(filters=16, activation="relu", kernel_size=3))
        model.add(MaxPooling2D(pool_size=2))
        model.add(
            Dropout(
                rate=hp.Float(
                    "dropout_1", min_value=0.0, max_value=0.5, default=0.25, step=0.05,
                )
            )
        )
        model.add(Conv2D(filters=32, kernel_size=3, activation="relu"))
        model.add(
            Conv2D(
                filters=hp.Choice("num_filters", values=[32, 64], default=64,),
                activation="relu",
                kernel_size=3,
            )
        )
        model.add(MaxPooling2D(pool_size=2))
        model.add(
            Dropout(
                rate=hp.Float(
                    "dropout_2", min_value=0.0, max_value=0.5, default=0.25, step=0.05,
                )
            )
        )
        model.add(Flatten())
        model.add(
            Dense(
                units=hp.Int(
                    "units", min_value=32, max_value=512, step=32, default=128
                ),
                activation=hp.Choice(
                    "dense_activation",
                    values=["relu", "tanh", "sigmoid"],
                    default="relu",
                ),
            )
        )
        model.add(
            Dropout(
                rate=hp.Float(
                    "dropout_3", min_value=0.0, max_value=0.5, default=0.25, step=0.05
                )
            )
        )
        model.add(Dense(self.num_classes, activation="softmax"))
    
        encoded_l=model(left_input)
        encoded_r=model(right_input)
        
        L1_layer = tf.keras.layers.Lambda(lambda tensors:K.abs(tensors[0]-tensors[1]))
        L1_distance=L1_layer([encoded_l,encoded_r])
        
        prediction=Dense(1,activation='sigmoid')(L1_distance)
    
        siamese_net1 = Model(inputs=[left_input,right_input],outputs=prediction)
        siamese_net1.compile(
            optimizer=keras.optimizers.Adam(
                hp.Float(
                    "learning_rate",
                    min_value=1e-4,
                    max_value=1e-2,
                    sampling="LOG",
                    default=1e-3,
                )
            ),
            loss="binary_crossentropy",
            metrics=["accuracy"])
        return siamese_net1
hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

In [0]:
# model = get_siamese_model((200,500,1))
# optimizer = Adam(lr=0.001)
# model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
# print("starting training process")
# print("...............................")
# #t_start = time.time()
# model.fit([x1_train,x2_train],y_train,verbose=1,batch_size=5,validation_data=[[x1_val,x2_val],y_val],epochs =20,shuffle = True)
# scores = model.evaluate([x1_test,x2_test],y_test,verbose=1)
# print(scores)
# model.save('siamese_net.h5')
# #t_stop = (time.time()-t_start())
# #print(t_stop)

In [0]:
import time

from kerastuner.tuners import (
    BayesianOptimization,
    Hyperband,
    RandomSearch,
)
from loguru import logger
from pathlib import Path

# from Hypermodels import CNNHyperModel
# from utils import (
#     set_gpu_config,
#     load_data,
# )

SEED = 1

NUM_CLASSES = 10
INPUT_SHAPE = (200,500,1)

N_EPOCH_SEARCH = 40
HYPERBAND_MAX_EPOCHS = 40
MAX_TRIALS = 20
EXECUTION_PER_TRIAL = 2
BAYESIAN_NUM_INITIAL_POINTS = 1
x_test, x_train, y_test, y_train =[x1_test,x2_test],[x1_train,x2_train],y_test,y_train

def run_hyperparameter_tuning():
    

    hypermodel = CNNHyperModel(input_shape=INPUT_SHAPE, num_classes=NUM_CLASSES)

    output_dir = Path("./output/cifar10/")
    tuners = define_tuners(
        hypermodel, directory=output_dir, project_name="simple_cnn_tuning"
    )

    results = []
    for tuner in tuners:
        elapsed_time, loss, accuracy = tuner_evaluation(
            tuner, x_test, x_train, y_test, y_train
        )
        logger.info(
            f"Elapsed time = {elapsed_time:10.4f} s, accuracy = {accuracy}, loss = {loss}"
        )
        results.append([elapsed_time, loss, accuracy])
    logger.info(results)


def tuner_evaluation(tuner, x_test, x_train, y_test, y_train):
    

    # Overview of the task
    tuner.search_space_summary()

    # Performs the hyperparameter tuning
    logger.info("Start hyperparameter tuning")
    search_start = time.time()
    tuner.search(x_train, y_train, epochs=N_EPOCH_SEARCH, validation_data=([x1_val,x2_val],y_val))
    search_end = time.time()
    elapsed_time = search_end - search_start

    # Show a summary of the search
    tuner.results_summary()

    # Retrieve the best model.
    best_model = tuner.get_best_models(num_models=1)[0]

    # Evaluate the best model.
    loss, accuracy = best_model.evaluate(x_test, y_test)
    return elapsed_time, loss, accuracy


def define_tuners(hypermodel, directory, project_name):
    random_tuner = RandomSearch(
        hypermodel,
        objective="val_accuracy",
        seed=SEED,
        max_trials=MAX_TRIALS,
        executions_per_trial=EXECUTION_PER_TRIAL,
        directory=f"{directory}_random_search",
        project_name=project_name,
    )
    hyperband_tuner = Hyperband(
        hypermodel,
        max_epochs=HYPERBAND_MAX_EPOCHS,
        objective="val_accuracy",
        seed=SEED,
        executions_per_trial=EXECUTION_PER_TRIAL,
        directory=f"{directory}_hyperband",
        project_name=project_name,
    )
    bayesian_tuner = BayesianOptimization(
        hypermodel,
        objective='val_accuracy',
        seed=SEED,
        num_initial_points=BAYESIAN_NUM_INITIAL_POINTS,
        max_trials=MAX_TRIALS,
        directory=f"{directory}_bayesian",
        project_name=project_name
    )
    return [random_tuner, hyperband_tuner, bayesian_tuner]

In [0]:
run_hyperparameter_tuning()


INFO:tensorflow:Reloading Oracle from existing project output/cifar10_random_search/simple_cnn_tuning/oracle.json


2020-06-03 10:11:04.418 | INFO     | __main__:tuner_evaluation:58 - Start hyperparameter tuning


Epoch 1/40
6/6 [==============================] - 57s 10s/step - loss: 0.6967 - accuracy: 0.5389 - val_loss: 0.6932 - val_accuracy: 0.3333
Epoch 2/40
6/6 [==============================] - 57s 10s/step - loss: 0.6941 - accuracy: 0.5333 - val_loss: 0.6933 - val_accuracy: 0.3333
Epoch 3/40
6/6 [==============================] - 56s 9s/step - loss: 0.6948 - accuracy: 0.5333 - val_loss: 0.6934 - val_accuracy: 0.3333
Epoch 4/40
6/6 [==============================] - 56s 9s/step - loss: 0.6892 - accuracy: 0.5278 - val_loss: 0.6935 - val_accuracy: 0.3333
Epoch 5/40
6/6 [==============================] - 56s 9s/step - loss: 0.7093 - accuracy: 0.5389 - val_loss: 0.6937 - val_accuracy: 0.3333
Epoch 6/40
6/6 [==============================] - 56s 9s/step - loss: 0.7028 - accuracy: 0.5167 - val_loss: 0.6938 - val_accuracy: 0.3333
Epoch 7/40
6/6 [==============================] - 56s 9s/step - loss: 0.6969 - accuracy: 0.5333 - val_loss: 0.6939 - val_accuracy: 0.3333
Epoch 8/40
6/6 [================

Epoch 1/40
6/6 [==============================] - 74s 12s/step - loss: 0.6941 - accuracy: 0.4667 - val_loss: 0.6931 - val_accuracy: 0.6667
Epoch 2/40
6/6 [==============================] - 69s 11s/step - loss: 0.6933 - accuracy: 0.5611 - val_loss: 0.6930 - val_accuracy: 0.6667
Epoch 3/40
6/6 [==============================] - 68s 11s/step - loss: 0.6942 - accuracy: 0.5167 - val_loss: 0.6930 - val_accuracy: 0.6667
Epoch 4/40
6/6 [==============================] - 69s 11s/step - loss: 0.6954 - accuracy: 0.4944 - val_loss: 0.6929 - val_accuracy: 0.6667
Epoch 5/40
6/6 [==============================] - 69s 11s/step - loss: 0.6888 - accuracy: 0.5500 - val_loss: 0.6929 - val_accuracy: 0.6667
Epoch 6/40
6/6 [==============================] - 69s 11s/step - loss: 0.6881 - accuracy: 0.5611 - val_loss: 0.6929 - val_accuracy: 0.6667
Epoch 7/40
6/6 [==============================] - 68s 11s/step - loss: 0.6909 - accuracy: 0.5000 - val_loss: 0.6928 - val_accuracy: 0.6667
Epoch 8/40
6/6 [===========

Epoch 1/40
6/6 [==============================] - 67s 11s/step - loss: 0.6944 - accuracy: 0.4722 - val_loss: 0.6931 - val_accuracy: 0.6667
Epoch 2/40
6/6 [==============================] - 65s 11s/step - loss: 0.6893 - accuracy: 0.5111 - val_loss: 0.6930 - val_accuracy: 0.6667
Epoch 3/40
6/6 [==============================] - 65s 11s/step - loss: 0.6933 - accuracy: 0.5333 - val_loss: 0.6931 - val_accuracy: 0.6667
Epoch 4/40
6/6 [==============================] - 65s 11s/step - loss: 0.6967 - accuracy: 0.5222 - val_loss: 0.6931 - val_accuracy: 0.6667
Epoch 5/40
6/6 [==============================] - 66s 11s/step - loss: 0.6963 - accuracy: 0.5111 - val_loss: 0.6931 - val_accuracy: 0.6667
Epoch 6/40
6/6 [==============================] - 65s 11s/step - loss: 0.6938 - accuracy: 0.5667 - val_loss: 0.6931 - val_accuracy: 0.6667
Epoch 7/40
6/6 [==============================] - 65s 11s/step - loss: 0.6939 - accuracy: 0.5111 - val_loss: 0.6931 - val_accuracy: 0.6667
Epoch 8/40
6/6 [===========